# Анализ пользовательского взаимодействия с карточками статей


**Автор:**  

Григорьев Павел


**Описание проекта:**   

Проект направлен на создание дашборда для визуализации данных, который упростит работу менеджеров Яндекс.Дзен, отвечая на их регулярные вопросы о взаимодействии пользователей с карточками статей. Дашборд будет автоматически подключаться к базе данных и визуализировать необходимую информацию.

**Цель:**  

Создать интерактивный дашборд для визуализации взаимодействия пользователей с карточками статей в Яндекс.Дзен.

**Источники данных:**  


Данные получены из внутренней системы Яндекс.Дзен.

**Главные выводы:**  
тут помещаем самое главное из общего вывода, примерно до полустраницы, чтобы не было сильно много и при этом указать все главные выводы
Будет идеально, елси выводы на похожие темы будут рядом, то есть елси мы имеем несколько выводов о доходе, то лушче поместить их рядом

- Женщины чаще возвращают кредит, чем мужчины.
- Долги присутствуют у людей с разным доходом.


**Рекомендации:**

- Добавить контроль данных, чтобы не дублировались значения с разными регистрами в колонке с образованием.
- Добавить уникальный идентификатор клиента, чтобы избежать дублирования строк.


**Дашборд:**

https://square-firefly-0750.ploomberapp.io/api/dashboard_for_yandex_afisha/

**Презентация:**

[В формате html](../../presentations/research_of_food_service_industry_presentation.html)  
[В формате pdf](../../presentations/research_of_food_service_industry_presentation.pdf)

## Составление технического задания 

Пообщавшись с менеджерами и администраторами баз данных, было составлено краткое ТЗ:
- Бизнес-задача: анализ взаимодействия пользователей с карточками Яндекс.Дзен;
- Насколько часто предполагается пользоваться дашбордом: не реже, чем раз в неделю;
- Кто будет основным пользователем дашборда: менеджеры по анализу контента;
- Состав данных для дашборда:
    - История событий по темам карточек (два графика - абсолютные числа и процентное соотношение);
    - Разбивка событий по темам источников;
    - Таблица соответствия тем источников темам карточек;
- По каким параметрам данные должны группироваться:
    - Дата и время;
    - Тема карточки;
    - Тема источника;
    - Возрастная группа;
- Характер данных:
    - История событий по темам карточек — абсолютные величины с разбивкой по минутам;
    - Разбивка событий по темам источников — относительные величины (% событий);
    - Соответствия тем источников темам карточек - абсолютные величины;
- Важность: все графики имеют равную важность;
- Источники данных для дашборда: cырые данные о событиях взаимодействия пользователей с карточками (таблица log_raw);
- База данных, в которой будут храниться агрегированные данные: дополнительные агрегированные таблицы в БД zen;
- Частота обновления данных: один раз в сутки, в полночь по UTC;

Макет дашборда:

![Макет дашборда](dashboard_scheme.png)

## Создание дашборда

Подключимся к  базе данных и загрузим данные.

In [4]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import plotly.express as px
import pagri_data_tools  # type: ignore
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:,.2f}'.format

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

db_config = {'user': 'praktikum_student', # имя пользователя
            'pwd': 'Sdf4$2;d-d30pp', # пароль
            'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
            'port': 6432, # порт подключения
            'db': 'data-analyst-zen-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                db_config['pwd'],
                                                db_config['host'],
                                                db_config['port'],
                                                db_config['db'])

engine = create_engine(connection_string) 

Чтобы посмотреть, какие таблицы есть в базе данных, вы можете использовать SQL-запрос для получения списка таблиц. В PostgreSQL это можно сделать с помощью запроса к системной таблице `information_schema.tables`.

In [2]:
# Запрос для получения списка таблиц
query = """
SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'public'
ORDER BY table_name;
"""

In [3]:
tables = pd.read_sql(query, engine)
tables

,table_name
0,dash_visits
1,dash_visits_new
2,log_raw


Нам нужна таблица `dash_visits`. Загрузим ее и сохраним в датафрейм.

In [6]:
df = pd.read_sql('select * from dash_visits', engine)
df.head()

,record_id,item_topic,source_topic,age_segment,dt,visits
0,1040597,Деньги,Авто,18-25,2019-09-24 18:32:00,3
1,1040598,Деньги,Авто,18-25,2019-09-24 18:35:00,1
2,1040599,Деньги,Авто,18-25,2019-09-24 18:54:00,4
3,1040600,Деньги,Авто,18-25,2019-09-24 18:55:00,17
4,1040601,Деньги,Авто,18-25,2019-09-24 18:56:00,27


Изучим визиты в разрезе тем карточек.

Агрегирурем даннные для построения графиков.  
Так как тем карточек и источников слшиком много, возьмем топ 10, а остальные поместим в категорию "другие".

In [7]:
top_item_topics = df.groupby('item_topic')['visits'].sum().sort_values(ascending=False).head(10).index.values
df['item_topic'] = df['item_topic'].apply(lambda x: x if x in top_item_topics else 'Другое')
df_aggregated_by_item_topic = df.groupby(['dt', 'item_topic'])[['visits']].sum()
df_aggregated_by_item_topic['all_visits'] = df_aggregated_by_item_topic.groupby('dt')['visits'].transform('sum')
df_aggregated_by_item_topic['visits_pct'] = df_aggregated_by_item_topic['visits'] * 100 / df_aggregated_by_item_topic['all_visits']
df_aggregated_by_item_topic = df_aggregated_by_item_topic.reset_index()
top_source_topic = df.groupby('source_topic')['visits'].sum().sort_values(ascending=False).head(10).index.values
# print(df.shape[0])
df['source_topic'] = df['source_topic'].apply(lambda x: x if x in top_source_topic else 'Другое')
df_aggregated_by_source_topic = (df.groupby('source_topic')[['visits']].sum() *100 / df.visits.sum()).sort_values('visits', ascending=False).reset_index()
df_aggregated_by_source_topic.rename(columns={'visits': 'visits_pct'}, inplace=True)

Создадим словарь для подписей графиков.

In [11]:
titles_for_axis = dict(
    visits = ['Количество визитов', 'количество визитов', 0]
    , visits_pct = ['Количество визитов, %', 'количество визитов, %', 0]
    # categorical column ['Именительный падеж', 'для кого / чего', 'по кому чему']
    # Распределение долей по городу и тарифу с нормализацией по городу
    , item_topic = ['Тема карточки', 'темы карточки', 'теме карточки']
    , source_topic = ['Тема источника', 'темы источника', 'теме источника']
    , age_segment = ['Возрастная группа', 'возрастная группа', 'возрастной группе']
    , dt = ['Дата', 'даты', 'дате']
)

In [12]:
config = dict(
    df = df_aggregated_by_item_topic
    , x = 'dt'  
    , y = 'visits'
    , category = 'item_topic'
    , width = None
    , height = None
    , orientation = 'v'
)
pagri_data_tools.area(config, titles_for_axis)

In [ ]:

config = dict(
    df = df_aggregated_by_item_topic
    , x = 'dt'  
    , y = 'visits_pct'
    , category = 'item_topic'
    , width = None
    , height = None
    , orientation = 'v'
)
# display(df_aggregated_by_source_topic)
visits_by_item_topic_pct_fig = pagri_data_tools.area(config, titles_for_axis)
visits_by_item_topic_pct_fig.update_layout(title_text=None, showlegend=False, margin=dict(l=0, r=0, b=0, t=0))    
config = dict(
    df = df_aggregated_by_source_topic
    , x = 'source_topic'  
    , y = 'visits_pct'
    , text = True
    , width = None
    , height = None
    , orientation = 'h'
    , showgrid_y = False
)
visits_by_source_topic_fig = pagri_data_tools.bar(config, titles_for_axis)
visits_by_source_topic_fig = visits_by_source_topic_fig.update_layout(title_text=None, showlegend=False, margin=dict(l=0, r=0, b=0, t=0))    
crosstab = df_origin.pivot_table(index = 'item_topic', columns = 'source_topic', values='visits', aggfunc='sum').fillna(0).astype(int).reset_index().rename(columns={'item_topic': ' '})
rowData = crosstab.to_dict('records')
table_component = dag.AgGrid(
    rowData=rowData, columnDefs=[{"field": i, 'headerName': i.replace('_', ' '), "cellStyle": {"fontSize": "14px"}, 'type': 'numeric', 'editable': False, "tooltipField": i, "minWidth": 100} if pd.api.types.is_numeric_dtype(crosstab[i])
                                    else {"field": i, 'headerName': i.replace('_', ' '), "cellStyle": {"fontSize": "14px"}, 'type': 'text', 'editable': False, "tooltipField": i, "minWidth": 100} for i in crosstab.columns]  # , 'type': 'numeric', "valueFormatter": {"function": "Number(params.value).toFixed(1)"}} , , 'autoSizeColumn': True
    # , columnSizeOptions = {"skipHeader": True}
    , columnSize="responsiveSizeToFit", defaultColDef={"sortable": True, "filter": False, "animateRows": True, "wrapHeaderText": True, "autoHeaderHeight": True}, dashGridOptions={"pagination": True, 'paginationPageSize': 10, "animateRows": True, "animateColumns": True}
    # ag-theme-quartz, ag-theme-quartz-dark, ag-theme-alpine, ag-theme-alpine-dark, ag-theme-balham, ag-theme-balham-dark, ag-theme-material.
)

In [ ]:
df.groupby(['dt', 'item_topic'])[['visits']].sum().reset_index()

visits
dt                  item_topic                
2019-09-24 18:28:00 Деньги                  24
                    Дети                    32
                    Женская психология      19
                    Женщины                 27
                    Здоровье                25
...                                        ...
2019-09-24 19:00:00 Семья                  681
                    Скандалы               507
                    Туризм                 538
                    Шоу                    408
                    Юмор                   504

[425 rows x 1 columns]

In [107]:
(table.groupby('source_topic')[['visits']].sum() *100 / table.visits.sum()).sort_values('visits', ascending=False).head()

,visits
source_topic,
Семейные отношения,10.737669
Россия,9.616482
Полезные советы,8.836680
Путешествия,7.776743
Знаменитости,7.719039


In [108]:
top_item_topics = table.groupby('item_topic')['visits'].sum().sort_values(ascending=False).head(10).index.values
table['item_topic'] = table['item_topic'].apply(lambda x: x if x in top_item_topics else 'Другое')
top_item_topics

array(['Наука', 'Отношения', 'Интересные факты', 'Общество', 'Подборки',
       'Россия', 'Полезные советы', 'История', 'Семья', 'Женщины'],
      dtype=object)

In [ ]:
table.pivot_table(index = 'item_topic', columns = 'source_topic', values='visits', aggfunc='sum').fillna(0).astype(int)